Case Studies

Project: 1

Group: 3

Group Members:
 - Muhammad Raafey Tariq (231806)
 - Farrukh Ahmed (230614)
 - Amirreza Khamehchin Khiabani (230891)
 - Aymane Hachcham (236392)


Requirements:
 - numpy==1.24.2
 - matplotlib==3.7.1
 - seaborn==0.12.2
 - pandas==2.0.0
 - openpyxl==3.1.2

Installation Commands (One-time only)
 - pip install pandas==2.0.0
 - pip install numpy==1.24.2
 - pip install seaborn==0.12.2
 - pip install matplotlib==3.7.1
 - pip install openpyxl==3.1.2

Imports and Libraries

In [276]:
import pandas as pd
import numpy as np
import itertools
import pprint

# used for the graphs
import seaborn as sns
import os
sns.set(font_scale = 1.2)

# used for plotting
from matplotlib import pyplot as plt
from datetime import datetime
import matplotlib

# setting font to 'Times New Roman'
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline

Global Variables and Constants

Importing Data

In [277]:
file_path = "../styrian_health_data.xlsx"
sheet_name = "Sheet 1"
data_df = pd.read_excel(file_path, sheet_name=sheet_name)

c:\Users\Raafe\AppData\Local\Programs\Python\Python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Reformating Columns to Correct Data Types, dropping nans and creating new variables

In [278]:
# fixing dtypes
data_df.postleitzahl = data_df.postleitzahl.astype('str')
data_df.geburtsjahr = data_df.geburtsjahr.astype('Int64')
# data_df.befinden = data_df.befinden.astype('Int64')
data_df.messwert_bp_sys = pd.to_numeric(data_df.messwert_bp_sys)
data_df.messwert_bp_dia = pd.to_numeric(data_df.messwert_bp_dia)
data_df.schaetzwert_bp_sys = pd.to_numeric(data_df.schaetzwert_bp_sys)
data_df.schaetzwert_by_dia = pd.to_numeric(data_df.schaetzwert_by_dia)

# adding variable for is_local
mask = data_df.gemeinde.isna() & data_df.bezirk.isna() & data_df.bundesland.isna()
data_df["is_local_resident"] = True
data_df.loc[mask, "is_local_resident"] = False

# adding variable for age
age =  data_df["zeit"].dt.year - pd.to_datetime(data_df['geburtsjahr'], format='%Y').dt.year
data_df["age"] = age.astype("Int64")

# adding variable for age group
data_df["age_group"] = pd.cut(data_df.age, bins=[0,12,19,65,130],labels=['children', 'teenager', 'adult','65 over'])
data_df["age_group"] = data_df.age_group.astype(str)

#replacing nans for variables

data_df.loc[data_df.geschlecht.isna() == True, 'raucher'] = "unknown"
data_df.loc[data_df.geschlecht.isna() == True, 'blutzucker_bekannt'] = "unknown"
data_df.loc[data_df.geschlecht.isna() == True, 'cholesterin_bekannt'] = "unknown"
data_df.loc[data_df.geschlecht.isna() == True, 'in_behandlung'] = "unknown"
data_df.loc[data_df.geschlecht.isna() == True, 'befinden'] = "unknown"
data_df.loc[data_df.age_group == "nan", 'age_group'] = "unknown"

data_df.loc[mask, 'gemeinde'] = "not applicable"
data_df.loc[mask, 'bezirk'] = "not applicable"
data_df.loc[mask, 'bundesland'] = "not applicable"
data_df.loc[mask, 'postleitzahl'] = "not applicable"
data_df.loc[data_df.postleitzahl == "nan", 'postleitzahl'] = "unknown"

# creating variables for missing values in bp

data_df["is_missing_schaetzwert_bp_sys"] = False
data_df.loc[data_df.schaetzwert_bp_sys.isna() == True, "is_missing_schaetzwert_bp_sys"] = True
data_df["is_missing_schaetzwert_by_dia"] = False
data_df.loc[data_df.schaetzwert_by_dia.isna() == True, "is_missing_schaetzwert_by_dia"] = True

# removing useless variables
data_df.drop(data_df[data_df.age > 100].index, inplace=True)
data_df.drop(data_df[data_df.age < 15].index, inplace=True)

data_df.loc[data_df.geschlecht.isna() == True, 'geschlecht'] = "unknown"

In [279]:
# Counts for missing values per column
data_df.isna().sum()

id                                0
zeit                              0
terminal                          0
postleitzahl                      0
gemeinde                          0
bezirk                            0
bundesland                        0
befinden                          0
geburtsjahr                      23
geschlecht                        0
raucher                           0
blutzucker_bekannt                0
cholesterin_bekannt               0
in_behandlung                     0
schaetzwert_bp_sys               27
schaetzwert_by_dia               35
messwert_bp_sys                   0
messwert_bp_dia                   0
is_local_resident                 0
age                              23
age_group                         0
is_missing_schaetzwert_bp_sys     0
is_missing_schaetzwert_by_dia     0
dtype: int64

In [280]:
len(data_df)

14887